In [1]:
import base64
import collections
import copy
import datetime
import itertools
import json
import math
import os
import operator
import pickle
import random
import re
import shutil
import sys
import time

import cv2
import Image
import keras
from keras import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL as pil
import pylab
import scipy
import seaborn as sns
import sklearn
from sklearn import *
import statsmodels as sm
import tensorflow as tf

np.random.seed(1337)

%matplotlib inline

sns.set(font_scale=1.3)
mpl.rcParams['figure.figsize'] = 20, 12
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

linewidth = 1.0
dotsize = 15

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/lo

In [2]:
base_dir = '/media/ntfs/data/invasive_species/'
input_dir = os.path.join(base_dir, 'input')
tmp_dir = os.path.join(base_dir, 'tmp')
output_dir = os.path.join(base_dir, 'output')

In [3]:
train_dir = os.path.join(tmp_dir, 'train')
val_dir = os.path.join(tmp_dir, 'val')
test_dir = os.path.join(input_dir, 'test')

In [4]:
train_pos_dir = os.path.join(train_dir, 'pos')
train_neg_dir = os.path.join(train_dir, 'neg')
val_pos_dir = os.path.join(val_dir, 'pos')
val_neg_dir = os.path.join(val_dir, 'neg')

In [5]:
model = keras.models.load_model('vgg16_01.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 13, 18, 512)       14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 119808)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               30671104  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 45,386,049
Trainable params: 37,750,785
Non-trainable params: 7,635,264
_________________________________________________________________


In [6]:
img_size = (433, 577)

In [7]:
# inv = []
# for i in range(1, 1532):
#     img = Image.open('/media/ntfs/data/invasive_species/input/test/foo/{0}.jpg'.format(i))
#     img = np.asarray(img.resize((577, 433)))
#     img.resize(1, 433, 577, 3)
#     inv.append(model.predict_proba(img, verbose=False)[0][0])
    
# names = range(1, 1532)

# s = pd.DataFrame({'name': names, 'invasive': inv})
# s = s[['name', 'invasive']]
# print s.head(20)
# s.to_csv(output_dir + '/s_02.csv', index=False)

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
                            test_dir,
                            target_size=img_size,
                            batch_size=1,
                            class_mode='binary',
                            shuffle=False)

invasive = model.predict_generator(test_generator, 1531)

Found 1531 images belonging to 1 classes.


In [15]:
names = [ int(n[4:][:-4]) for n in test_generator.filenames]

In [16]:
s = pd.DataFrame({'name': names, 'invasive': invasive.reshape(1531)}).sort_values('name')
s = s[['name', 'invasive']]
print s.head(20)
s.to_csv(output_dir + '/s_03.csv', index=False)

      name      invasive
2        1  1.000000e+00
632      2  3.166607e-03
738      3  2.745320e-02
861      4  1.549785e-03
967      5  1.000000e+00
1073     6  2.403014e-03
1179     7  1.476618e-02
1318     8  1.000000e+00
1424     9  1.000000e+00
3       10  8.245123e-05
127     11  2.981808e-06
232     12  9.921021e-03
338     13  2.277796e-05
443     14  1.000000e+00
549     15  1.268112e-24
590     16  7.360006e-03
600     17  3.255603e-03
611     18  3.546090e-05
621     19  1.094942e-23
633     20  3.592192e-03
